In [ ]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf

!pip -q install sentence_transformers
!pip install chromadb

!pip install openai
!pip install tiktoken

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from huggingface_hub import notebook_login
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

import os
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

directory_path = '/content/drive/MyDrive/docs1'



In [ ]:
files = os.listdir(directory_path)

In [ ]:
document=[]
for file in files:

  if file.endswith(".pdf"):
    pdf_path=""+file
    loader=PyPDFLoader(pdf_path)
    document.extend(loader.load())
  elif file.endswith('.docx') or file.endswith('.doc'):
    doc_path=""+file
    loader=Docx2txtLoader(doc_path)
    document.extend(loader.load())
  elif file.endswith('.txt'):
    text_path = "/content/drive/MyDrive/docs1/"+file

    loader=TextLoader(text_path, encoding = 'UTF-8')
    document.extend(loader.load())

In [ ]:
# document_splitter=CharacterTextSplitter(separator='\n', chunk_size=500, chunk_overlap=100)
document_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, separators=[" ", ",", "\n"])

In [ ]:
document_chunks=document_splitter.split_documents(document)

In [ ]:
len(document_chunks)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
os.environ["OPENAI_API_KEY"]="s"
embeddings = OpenAIEmbeddings()

In [ ]:
vectordb=Chroma.from_documents(document_chunks,embedding=embeddings, persist_directory='./data')

In [ ]:
vectordb.persist()

Llama

In [ ]:
notebook_login()

In [ ]:
##llama

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              #load_in_8bit=True,
                                              load_in_4bit=True
                                             )

In [ ]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map='auto',
              max_new_tokens=512,
              min_new_tokens=-1,
              top_k=30

              )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

GPT

In [ ]:
llm=ChatOpenAI(temperature=0.7, model_name='gpt-4')

In [ ]:
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [ ]:
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                             retriever=vectordb.as_retriever(search_kwargs={'k':20}),
                                             verbose=False, memory=memory)

In [ ]:
result=pdf_qa({"question":"Given the following ontology, extract the related parts of the documents match with the ontology, ontology:has_Name(Agent, Name), has_Surname(Agent, Surname), has_First_Name(Agent, First_Name), has_Alternate_Name(Agent, Alternate_Name), recorded_At(Agent_name, Event), recorded_At(Agent_First_Name, Event), recorded_At(Agent_Surname, Event), recorded_At(Agent_Alternate_Name, Event). geerate texts to populate  based on the relations and document"})

In [ ]:
result['answer']

In [ ]:
file_path = "content/drive/MyDrive/outputn.txt"

# Open the file in write mode
with open(file_path, "w", encoding="utf-8") as file:
    # Write the variable's value to the file
    file.write(result['answer'])
